In [2]:
import os
import sys
sys.path.append('../')
import util.data_processing_tool as dpt
from model import vdsr

from datetime import timedelta, date, datetime
# import args_parameter as args
import torch,torchvision
import numpy as np
import random
import properscoring as ps
from torch.utils.data import Dataset,random_split
from torchvision import datasets, models, transforms

import time
import xarray as xr
from PIL import Image


import matplotlib as plt
import argparse
import sys
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, models, transforms
import platform
from datetime import timedelta, date, datetime

import torch.nn as nn

import torch.optim as optim

from math import log10
import time
# from PrepareData import ACCESS_BARRA_crps
import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
# ===========================================================
# Training settings
# ===========================================================


def rmse(ens,hr):
    '''
    ens:(ensemble,H,W)
    hr: (H,W)
    '''
    return np.sqrt((ens-hr).sum(axis=(0)))

def mae(ens,hr):
    '''
    ens:(ensemble,H,W)
    hr: (H,W)
    '''
    return np.abs((ens-hr)).sum(axis=0)


class ACCESS_BARRA_crps(Dataset):
    '''

2.using my net to train one channel to one channel.
   
    '''
    def __init__(self,start_date=date(1990, 1, 1),end_date=date(1990,12 , 31),regin="AUS",lr_transform=None,hr_transform=None,shuffle=True,args=None):
#         print("=> BARRA_R & ACCESS_S1 loading")
#         print("=> from "+start_date.strftime("%Y/%m/%d")+" to "+end_date.strftime("%Y/%m/%d")+"")
        self.file_BARRA_dir = args.file_BARRA_dir
        self.file_ACCESS_dir = args.file_ACCESS_dir
        self.args=args
        
        self.lr_transform = lr_transform
        self.hr_transform = hr_transform

        self.start_date = start_date
        self.end_date = end_date
        
        self.regin = regin
        self.leading_time_we_use=args.leading_time_we_use

        self.ensemble_access=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']
        self.ensemble=[]
        for i in range(args.ensemble):
            self.ensemble.append(self.ensemble_access[i])
                
        self.dates = self.date_range(start_date, end_date)
        
        
        self.filename_list=self.get_filename_with_time_order(args.file_ACCESS_dir+"pr/daily/")
#         if not os.path.exists(args.file_ACCESS_dir+"pr/daily/"):
#             print(args.file_ACCESS_dir+"pr/daily/")
#             print("no file or no permission")
        
        
        _,_,date_for_BARRA,time_leading=self.filename_list[0]
        if shuffle:
            random.shuffle(self.filename_list)
        
        
#         if not os.path.exists("/g/data/ma05/BARRA_R/v1/forecast/spec/accum_prcp/1990/01/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19900109T0600Z.sub.nc"):
#             print(self.file_BARRA_dir)
#             print("no file or no permission!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        data_high=dpt.read_barra_data_fc_get_lat_lon(self.file_BARRA_dir,date_for_BARRA)
        self.lat=data_high[1]
        self.lon=data_high[1]
#         self.shape=(316, 376)


#         print(type(self.data_dem))
        
#             data_dem=dpt.add_lat_lon( dpt.read_dem(args.file_DEM_dir+"dem-9s1.tif"))
#             self.dem_data=dpt.interp_tensor_2d(dpt.map_aust_old(data_dem,xrarray=False) ,self.shape )
        

        
    def __len__(self):
        return len(self.filename_list)
    

    def date_range(self,start_date, end_date):
        """This function takes a start date and an end date as datetime date objects.
        It returns a list of dates for each date in order starting at the first date and ending with the last date"""
        return [start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]

    
    def get_filename_with_no_time_order(self,rootdir):
        '''get filename first and generate label '''
        _files = []
        list = os.listdir(rootdir) #åˆ—å‡ºæ–‡ä»¶å¤¹ä¸‹æ‰€æœ‰çš„ç›®å½•ä¸Žæ–‡ä»¶
        for i in range(0,len(list)):
            path = os.path.join(rootdir,list[i])
            if os.path.isdir(path):
                _files.extend(self.get_filename_with_no_time_order(path))
            if os.path.isfile(path):
                if path[-3:]==".nc":
                    _files.append(path)
        return _files
    
    def get_filename_with_time_order(self,rootdir):
        '''get filename first and generate label ,one different w'''
        _files = []
        for date in self.dates:
            for i in range(self.leading_time_we_use,self.leading_time_we_use+1):
            

#                 filename="da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"cd
#                 access_path=rootdir+en+"/"+"da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"
#                 print(access_path)
                for en in self.ensemble:
                    access_path=rootdir+en+"/"+"da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"
                    if os.path.exists(access_path):
                        
                    
                        if date==self.end_date and i==1:
                            break
                        path=[]
                        path.append(en)
                        barra_date=date+timedelta(i)
                        path.append(date)
                        path.append(barra_date)
                        path.append(i)
                        _files.append(path)

#     def get_filename_with_time_order(self,rootdir):
#         '''get filename first and generate label ,one different w'''
#         _files = []
#         for date in self.dates:
#             for en in self.ensemble:

# #                 filename="da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"cd
#                 access_path=rootdir+en+"/"+"da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"
# #                 print(access_path)
#                 if os.path.exists(access_path):
#                     for i in range(self.leading_time_we_use):
#                         if date==self.end_date and i==1:
#                             break
#                         path=[]
#                         path.append(en)
#                         barra_date=date+timedelta(i)
#                         path.append(date)
#                         path.append(barra_date)
#                         path.append(i)
#                         _files.append(path)                        

    #æœ€åŽåŽ»æŽ‰ç¬¬ä¸€è¡Œï¼Œç„¶åŽshuffle
        return _files

    def mapping(self,X,min_val=0.,max_val=255.):
        Xmin = np.min(X)
        Xmax = np.max(X)
        #å°†æ•°æ®æ˜ å°„åˆ°[-1,1]åŒºé—´ å³a=-1ï¼Œb=1
        a = min_val
        b = max_val
        Y = a + (b-a)/(Xmax-Xmin)*(X-Xmin)
        return Y
        
    def __getitem__(self, idx):
        '''
        from filename idx get id
        return lr,hr
        '''
        t=time.time()
#         data[var_name][:7,82:144,134:188]*86400
        #read_data filemame[idx]
        en,access_date,barra_date,time_leading=self.filename_list[idx]
        
        lr=dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"pr")*86400

#         lr=np.expand_dims(lr,axis=2)
        lr=dpt.interp_tensor_2d(lr,(79,94))
#         lr=np.expand_dims(self.mapping(lr),axis=2)
        label=dpt.read_barra_data_fc(self.file_BARRA_dir,barra_date)

        if self.args.zg:
            lr_zg=np.expand_dims(dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"zg"),axis=2)

        if self.args.psl:
            lr_psl=dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"psl")

        if self.args.tasmax:
            lr_tasmax=np.expand_dims(dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"tasmax"),axis=2)


        if self.args.tasmin:
            lr_tasmin=dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"tasmin")
            
#         if self.args.channels==1:
#             lr=np.repeat(lr,3,axis=2)
#         return self.lr_transform(Image.fromarray(lr)),self.lr_transform(Image.fromarray(self.data_dem)),self.hr_transform(Image.fromarray(label))
        
        return self.lr_transform(Image.fromarray(lr)),torch.tensor(int(en[1:])),self.hr_transform(Image.fromarray(label)),torch.tensor(int(en[1:])),torch.tensor(int(access_date.strftime("%Y%m%d"))),torch.tensor(time_leading)




def write_log(log,args):
    print(log)
    if not os.path.exists("./save/"+args.train_name+"/"):
        os.mkdir("./save/"+args.train_name+"/")
    my_log_file=open("./save/"+args.train_name + '/train.txt', 'a')
#     log="Train for batch %d,data loading time cost %f s"%(batch,start-time.time())
    my_log_file.write(log + '\n')
    my_log_file.close()
    return
 
def main(year):
    parser = argparse.ArgumentParser(description='PyTorch Super Res Example')
    # Hardware specifications
    parser.add_argument('--n_threads', type=int, default=0,
                        help='number of threads for data loading')

    parser.add_argument('--cpu', action='store_true',help='cpu only?') 

    # hyper-parameters
    parser.add_argument('--train_name', type=str, default="cali_crps", help='training name')

    parser.add_argument('--batch_size', type=int, default=44, help='training batch size')
    parser.add_argument('--testBatchSize', type=int, default=4, help='testing batch size')
    parser.add_argument('--nEpochs', type=int, default=200, help='number of epochs to train for')
    parser.add_argument('--lr', type=float, default=0.0001, help='Learning Rate. Default=0.01')
    parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')

    # model configuration
    parser.add_argument('--upscale_factor', '-uf',  type=int, default=4, help="super resolution upscale factor")
    parser.add_argument('--model', '-m', type=str, default='vdsr', help='choose which model is going to use')

    #data
    parser.add_argument('--pr', type=bool, default=True,help='add-on pr?')

    parser.add_argument('--train_start_time', type=type(datetime(1990,1,25)), default=datetime(1990,1,2),help='r?')
    parser.add_argument('--train_end_time', type=type(datetime(1990,1,25)), default=datetime(1990,2,9),help='?')
    parser.add_argument('--test_start_time', type=type(datetime(2012,1,1)), default=datetime(2010,1,1),help='a?')
    parser.add_argument('--test_end_time', type=type(datetime(2012,12,31)), default=datetime(2010,12,31),help='')

    parser.add_argument('--dem', action='store_true',help='add-on dem?') 
    parser.add_argument('--psl', action='store_true',help='add-on psl?') 
    parser.add_argument('--zg', action='store_true',help='add-on zg?') 
    parser.add_argument('--tasmax', action='store_true',help='add-on tasmax?') 
    parser.add_argument('--tasmin', action='store_true',help='add-on tasmin?')
    parser.add_argument('--leading_time_we_use', type=int,default=1
                        ,help='add-on tasmin?')
    parser.add_argument('--ensemble', type=int, default=11,help='total ensambles is 11') 
    parser.add_argument('--channels', type=float, default=0,help='channel of data_input must') 
    #[111.85, 155.875, -44.35, -9.975]
    parser.add_argument('--domain', type=list, default=[112.9, 154.25, -43.7425, -9.0],help='dataset directory')

#     parser.add_argument('--file_ACCESS_dir', type=str, default="/g/data/ub7/access-s1/hc/raw_model/atmos/",help='dataset directory')
    parser.add_argument('--file_ACCESS_dir', type=str, default="../../Data/",help='dataset directory')

    parser.add_argument('--file_BARRA_dir', type=str, default="../../Data/barra_aus/",help='dataset directory')
    parser.add_argument('--file_DEM_dir', type=str, default="../DEM/",help='dataset directory')
    parser.add_argument('--precision', type=str, default='single',choices=('single', 'half','double'),help='FP precision for test (single | half)')

    args = parser.parse_args([])
    
    # def main():

#     init_date=date(1970, 1, 1)
#     start_date=date(1990, 1, 2)
#     end_date=date(2011,12,25)
    sys = platform.system()
    args.dem=False
    args.train_name="pr_dem"
    args.channels=0
    if args.pr:
        args.channels+=1
    if args.zg:
        args.channels+=1
    if args.psl:
        args.channels+=1
    if args.tasmax:
        args.channels+=1
    if args.tasmin:
        args.channels+=1
    if args.dem:
        args.channels+=1
    print("training statistics:")
    print("  ------------------------------")
    print("  trainning name  |  %s"%args.train_name)
    print("  ------------------------------")
    print("  num of channels | %5d"%args.channels)
    print("  ------------------------------")
    print("  num of threads  | %5d"%args.n_threads)
    print("  ------------------------------")
    print("  batch_size     | %5d"%args.batch_size)
    print("  ------------------------------")
    print("  using cpu only | %5d"%args.cpu)

    lr_transforms = transforms.Compose([
        transforms.Resize((316, 376)),
    #     transforms.RandomResizedCrop(IMG_SIZE),
    #     transforms.RandomHorizontalFlip(),
    #     transforms.RandomRotation(30),
        transforms.ToTensor()
    #     transforms.Normalize(IMG_MEAN, IMG_STD)
    ])

    hr_transforms = transforms.Compose([
    #         transforms.Resize((316, 376)),
    #     transforms.RandomResizedCrop(IMG_SIZE),
    #     transforms.RandomHorizontalFlip(),
    #     transforms.RandomRotation(30),
        transforms.ToTensor()
    #     transforms.Normalize(IMG_MEAN, IMG_STD)
    ])
    args.test_start_time=datetime(year,1,1)
    args.test_end_time=datetime(year,12,31)
    data_set=ACCESS_BARRA_crps(args.test_start_time,args.test_end_time,lr_transform=lr_transforms,hr_transform=hr_transforms,shuffle=False,args=args)



    #     #######################################################################

    test_data=DataLoader(data_set,
                                            batch_size=args.batch_size,
                                            shuffle=False,
                                num_workers=args.n_threads,drop_last=True)

    #     #######################################################################


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    if year==2012:
        net=torch.load('../save/model/vdsr_pr/best_test.pth',map_location=device)
    else:
        net=torch.load('../save/model/vdsr_pr/best_test.pth',map_location=device)['model']

    def prepare( l, device=False):
        def _prepare(tensor):
            if args.precision == 'half': tensor = tensor.half()
            if args.precision == 'single': tensor = tensor.float()
            return tensor.to(device)

        return [_prepare(_l) for _l in l]


    #     ##############################################

    #     max_error=np.inf
    #     val_max_error=np.inf

    #     print(data_set.filename_list)

    # for e in range(args.nEpochs):
    #         loss=0
    for lead in range(217):
        args.leading_time_we_use=lead

        data_set=ACCESS_BARRA_crps(args.test_start_time,args.test_end_time,lr_transform=lr_transforms,hr_transform=hr_transforms,shuffle=False,args=args)


        test_data=DataLoader(data_set,
                                                batch_size=args.batch_size,
                                                shuffle=False,
                                    num_workers=args.n_threads,drop_last=False)


        crps_score_vsdr=[]
        mae_score_vsdr=[]
        rmse_score_vsdr=[]
        start=time.time()
        fmt = '%Y%m%d'


    #         test_data=tqdm.tqdm(test_data)
        for batch, (pr,dem,hr,en,data_time,idx) in enumerate(test_data):
            pr,dem,hr= prepare([pr,dem,hr],device)
            with torch.set_grad_enabled(False):

#                 sr = net(pr)
                sr_np=pr.cpu().numpy()
                hr_np=hr.cpu().numpy()
#                 print(pr.shape)
#                 print(hr.shape)
#                 print(en)
#                 print(data_time)
                for i in range(args.batch_size//args.ensemble):
                    a=np.squeeze( sr_np[i*args.ensemble:(i+1)*args.ensemble])
                    b=np.squeeze(hr_np[i*args.ensemble])
                    print(a.shape,b.shape)
                    #skil=vectcrps_m(a,b)
#                    skil=ps.crps_ensemble(b,np.transpose(a,(1,2,0)))
                    rmes_score=rmse(a,b)
                    mae_score=mae(a,b)




                    #crps_score_vsdr.append(skil)
                    mae_score_vsdr.append(mae_score)
                    rmse_score_vsdr.append(rmes_score)
        if not os.path.exists("../save/rmse/vdsrd/"+str(year)):
            dpt.mkdir("../save/rmse/vdsrd/"+str(year))
            
        if not os.path.exists("../save/mae/vdsrd/"+str(year)):
            dpt.mkdir("../save/mae/vdsrd/"+str(year))
            
        if not os.path.exists("../save/crps/vdsrd/"+str(year)):
            dpt.mkdir("../save/crps/vdsrd/"+str(year))
#         np.save("../save/crps/bi_217/2010/lead_time_"+str(lead),crps_score_vsdr)
        np.save("../save/rmse/vdsrd/"+str(year)+"/lead_time_"+str(lead),rmse_score_vsdr)
        np.save("../save/mae/vdsrd/"+str(year)+"/lead_time_"+str(lead),mae_score_vsdr)
        
        print(str(lead)+" : "+str(np.array(rmse_score_vsdr).mean()),",")
        print(str(lead)+" : "+str(np.array(mae_score_vsdr).mean()),",")
            


if __name__=='__main__':
#     main(year=1997) 
#     print('1997done')
#     main(year=2010) 
#     print('2010done')
    main(year=2012)
    print('2012done')
  


training statistics:
  ------------------------------
  trainning name  |  pr_dem
  ------------------------------
  num of channels |     1
  ------------------------------
  num of threads  |     0
  ------------------------------
  batch_size     |    44
  ------------------------------
  using cpu only |     0
(11, 316, 376) (316, 376)
(11, 316, 376) (316, 376)
(11, 316, 376) (316, 376)
(11, 316, 376) (316, 376)
(11, 316, 376) (316, 376)
(11, 316, 376) (316, 376)
(11, 316, 376) (316, 376)
(11, 316, 376) (316, 376)


KeyboardInterrupt: 

In [21]:
dpt.read_access_data('../../Data/','e01',date(2012,1,1),0).shape

(79, 94)

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net=torch.load('../save/model/vdsr_pr/best_test.pth',map_location=device)

In [ ]:
import sys
sys.path.append('../')
import util.PrepareData as pdata
import util.data_processing_tool as dpt

from datetime import date
from netCDF4 import Dataset
import numpy as np
import netCDF4 as nc
from datetime import timedelta, date, datetime
import platform 
import torch
from torchvision import datasets, models, transforms
import os
from PIL import Image
import util.constant_param as consparam
start_date=date(2012, 1, 1)
end_date=date(2012,12,25)
dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]



def get_filename_with_no_time_order(rootdir):
    '''get filename first and generate label '''
    _files = []
    list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
    for i in range(0,len(list)):
        path = os.path.join(rootdir,list[i],)
        if os.path.isdir(path):
            _files.extend(get_filename_with_no_time_order(path))
        if os.path.isfile(path):
            if path[-3:]==".nc":
                _files.append(path)
    return _files


def get_filename_with_time_order(rootdir,ensemble,dates,var_name):
    '''get filename first and generate label ,one different w'''
    _files = []
    for en in ensemble:
        for date in dates:
            access_path=rootdir+var_name+'/daily/'+en+"/"+"da_"+var_name+"_"+date.strftime("%Y%m%d")+"_"+en+".nc"
#             print(access_path)
            if os.path.exists(access_path):
#                 print(access_path)
                path=[access_path]
                path.append(en)
                path.append(date)
                _files.append(path)

#最后去掉第一行，然后shuffle
#     if nine2nine and date_minus_one==1:
#         del _files[0]
    return _files


#50 station
def get_access_SR_data():# 顺便保存 BI数据
    
    
    lr_transforms = transforms.Compose([
    transforms.Resize((316, 376)),
#     transforms.RandomResizedCrop(IMG_SIZE),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(30),
    transforms.ToTensor()
#     transforms.Normalize(IMG_MEAN, IMG_STD)
    ])
    
    
    import netCDF4 as nc
#     ensemble=['e01','e02']
    sys = platform.system()
    init_date=date(1970, 1, 1)
    start_date=date(2012, 1, 1)
    end_date=date(2012,12,25) #if 929 is true we should substract 1 day  
    if sys == "Windows":
        init_date=date(1970, 1, 1)
        start_date=date(1990, 1, 1)
        end_date=date(2012,12,25) #if 929 is true we should substract 1 day   
        file_ACCESS_dir="H:/climate/access-s1/" 
        file_BARRA_dir="D:/dataset/accum_prcp/"
    #         args.file_ACCESS_dir="E:/climate/access-s1/"
    #         args.file_BARRA_dir="C:/Users/JIA059/barra/"
        file_DEM_dir="../DEM/"
    else:
        file_ACCESS_dir_pr="/g/data/ub7/access-s1/hc/raw_model/atmos/pr/daily/"
        file_ACCESS_dir="/g/data/ub7/access-s1/hc/raw_model/atmos/"
        # training_name="temp01"
        file_BARRA_dir="/g/data/ma05/BARRA_R/v1/forecast/spec/accum_prcp/"

        
    ensemble=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']

    var_name="pr"
    dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
    file_list=get_filename_with_time_order(file_ACCESS_dir+var_name+'/daily/',ensemble,dates,var_name)
    time_leading=217

    lat_name="lat"
    lon_name="lon"

#     print(file_list)
    for i in file_list:
#         print(i)
        data = Dataset(i[0], 'r')
        var = data[var_name][:,82:144,134:188]*86400
        lat = data[lat_name][:][82:144]
        lon = data[lon_name][:][134:188]
#         print(var.shape)
        data.close()
    #         lr=dpt.read_access_data(i,idx=time_leading).data[82:144,134:188]*86400
        result= torch.zeros((217,1,316,376),dtype=torch.float32)

        for idx,j in enumerate(var):
            if idx>= 217:
                break
            lr=dpt.interp_tensor_2d(j,(79,94))
            result[idx]=lr_transforms(Image.fromarray(lr))
            


        
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        net=torch.load('../save/model/vdsr_pr/best_test.pth',map_location=device)
        def prepare( l, device=False):
            def _prepare(tensor):
                tensor = tensor.float()
                return tensor.to(device)
            return [_prepare(_l) for _l in l]
        llrr=prepare([result],device)[0]
        with torch.set_grad_enabled(False):
            sr=net(llrr)
        sr_np=np.squeeze(sr.cpu().numpy())
        
        data_50=[]
        station_name=[]
        for station in consparam.station_50_index_for_size_of_hr_sr_station_code.keys():
            idx_i=consparam.station_50_index_for_size_of_hr_sr_station_code[station][0]
            idx_j=consparam.station_50_index_for_size_of_hr_sr_station_code[station][1]
            data_50.append(sr_np[:,idx_i,idx_j])
            station_name.append(station)
        data_50=np.array(data_50)
        print(data_50.shape)
        station_name=np.array(station_name)
        
        if not os.path.exists('../Data/'+var_name+'/daily_vdsrd_50station/'+i[1]):
            os.makedirs('../Data/'+var_name+'/daily_vdsrd_50station/'+i[1])
        
        f_w = nc.Dataset('../Data/'+var_name+'/daily_vdsrd_50station/'+i[1]+"/da_"+var_name+"_"+i[2].strftime("%Y%m%d")+"_"+i[1]+'.nc','w',format = 'NETCDF4')

        f_w.createDimension('time',time_leading)
        f_w.createDimension('station',50)

        f_w.createVariable('station',np.int32,('station'))
        f_w.createVariable('time',np.int,('time'))



        f_w.variables['station'][:] = station_name
        f_w.variables['time'][:] =  np.array(range(1,218))


        f_w.createVariable( 'pr', np.float32, ('station','time'))
        f_w.variables['pr'][:] = data_50

        f_w.close()
            

            


In [ ]:
if __name__=='__main__':
    get_access_SR_data()